In [72]:
import pandas as pd

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url, attrs={'class': 'sortable'}, header=0);
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
myWorkingDF = df[0]
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
myWorkingDF = myWorkingDF[myWorkingDF.Borough != 'Not assigned']
#More than one neighborhood can exist in one postal code area.
myWorkingDF = myWorkingDF.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
myWorkingDF.loc[myWorkingDF['Neighbourhood']=='Not assigned', 'Neighbourhood']=myWorkingDF['Borough']
myWorkingDF.shape

(103, 3)